In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/testing/WhatsApp Image 2025-05-29 at 20.28.26_297b7a2f.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0245.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0250.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0277.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0239.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0229.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0208.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0204.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0223.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0243.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0227.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0287.jpg
/kaggle/input/computer-vision-info-educatie/yellow/IMG-20250529-WA0270.jpg
/kaggle/input/computer-visi

In [80]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import cv2
import os
from tqdm import tqdm
from sklearn.metrics import classification_report

In [81]:
path = '/kaggle/input/computer-vision-info-educatie'

In [82]:
def folder_to_label(color):
    if color == 'yellow':
        return 0
    elif color == 'blue':
        return 0
    elif color == 'green':
        return 0
    return 1

In [83]:
class data(Dataset):
    def __init__(self, path):
        self.images = []
        self.label = []

        self.transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
            transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=1),
            transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        for folder in os.listdir(path):
            for img in os.listdir(os.path.join(path, folder)):
                self.images.append(os.path.join(path, folder, img))
                self.label.append(folder_to_label(folder))
                
    def __len__(self):
        return len(self.label)
        
    def __getitem__(self,index):
        input_img = self.transform(Image.open(self.images[index]).convert('RGB'))
        output_label = torch.tensor(self.label[index])
        return input_img, output_label    

In [84]:
dataset = data(path)

In [85]:
train_dataset, valid_dataset, test_dataset = random_split(dataset, [int(0.8 * len(dataset)), int(0.1 * len(dataset)), len(dataset) - int(0.1 * len(dataset)) - int(0.8 * len(dataset))])
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [86]:
def Linear(input_features, output_features):
    layer = nn.Sequential(
        nn.Linear(input_features, output_features),
        nn.ReLU(),
        nn.Dropout(0.4)
    )
    return layer

In [87]:
def Conv(input_features, output_features, kernel_sz):
    convolution = nn.Sequential(
        nn.Conv2d(in_channels=input_features, out_channels=output_features, kernel_size=kernel_sz, stride = 2),
        nn.BatchNorm2d(output_features),
        nn.ReLU(),
        nn.MaxPool2d((2,2), 2),
        nn.Dropout2d(0.4)
    )
    return convolution

In [88]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [89]:
from torchvision.models import vit_b_16, ViT_B_16_Weights

In [90]:
weights = ViT_B_16_Weights.DEFAULT
model = vit_b_16(weights=weights)

in_features = model.heads[0].in_features

model.heads = nn.Sequential(nn.Linear(in_features, 2))
model = model.to(device)

In [91]:
weight = torch.tensor([1.0, 253 / 78], dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-2)
softmax = nn.Softmax(dim=-1)
epochs = 8

In [92]:
import matplotlib.pyplot as plt

train_losses = []
valid_losses = []

In [ ]:
for epoch in range(epochs):
    model.train()
    predictions = []
    ground_truth = []
    running_loss = 0.0

    dataloader = tqdm(train_dataloader, desc=f"Training on epoch {epoch + 1}...", leave=False)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_hat = model(X)
        loss = loss_fn(y_hat, y)
        running_loss += loss.item()

        preds = torch.argmax(softmax(y_hat), dim=1)
        predictions.extend(preds.cpu().numpy())
        ground_truth.extend(y.cpu().numpy())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_train_loss = running_loss / len(train_dataloader)
    train_losses.append(epoch_train_loss)
    print(f"Training epoch {epoch + 1} - Loss: {epoch_train_loss:.4f}")
    print("Classification Report (Train):")
    print(classification_report(ground_truth, predictions))

    # Evaluation
    model.eval()
    predictions = []
    ground_truth = []
    running_loss = 0.0

    with torch.no_grad():
        dataloader = tqdm(valid_dataloader, desc=f"Validating on epoch {epoch + 1}...", leave=False)
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            y_hat = model(X)
            loss = loss_fn(y_hat, y)
            running_loss += loss.item()

            preds = torch.argmax(softmax(y_hat), dim=1)
            predictions.extend(preds.cpu().numpy())
            ground_truth.extend(y.cpu().numpy())

    epoch_valid_loss = running_loss / len(valid_dataloader)
    valid_losses.append(epoch_valid_loss)
    print(f"Validation epoch {epoch + 1} - Loss: {epoch_valid_loss:.4f}")
    print("Classification Report (Validation):")
    print(classification_report(ground_truth, predictions))

Training on epoch 1...:  18%|█▊        | 2/11 [00:03<00:14,  1.59s/it]

In [ ]:
# Plot after training
plt.figure(figsize=(10, 5))
plt.plot(range(1, 6), train_losses, label='Training Loss', marker='o')
plt.plot(range(1, 6), valid_losses, label='Validation Loss', marker='x')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss per Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model.eval()
predictions = []
ground_truth = []
running_loss = 0.0

with torch.no_grad():
    dataloader = tqdm(test_dataloader, desc=f"Testing on epoch {epoch + 1}...", leave=False)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_hat = model(X)
        loss = loss_fn(y_hat, y)
        running_loss += loss.item()

        preds = torch.argmax(softmax(y_hat), dim=1)
        predictions.extend(preds.cpu().numpy())
        ground_truth.extend(y.cpu().numpy())

running_loss /= len(test_dataloader)
print(f"Validation epoch {epoch + 1} - Loss: {running_loss:.4f}")
print("Classification Report (Testing):")
print(classification_report(ground_truth, predictions))

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
img_path = "/kaggle/input/testing5/WhatsApp Image 2025-05-29 at 22.16.34_288d61ff.jpg"
image = Image.open(img_path).convert('RGB')

transformed_image = transform(image)
transformed_image = transformed_image.unsqueeze(0)
transformed_image = transformed_image.to(device)

In [ ]:
with torch.no_grad():
    output = model(transformed_image)
    probs = torch.softmax(output, dim=1)
    print(probs)
    predicted_class = torch.argmax(probs, dim=1).item()

In [ ]:
def color_to_label(color):
    if color == 'yellow':
        return 0
    elif color == 'blue':
        return 1
    return 2

In [ ]:
class data_color(Dataset):
    def __init__(self, path):
        self.images = []
        self.label = []

        self.transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
            transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        for folder in os.listdir(path):
            if folder != 'fake':
                for img in os.listdir(os.path.join(path, folder)):
                    self.images.append(os.path.join(path, folder, img))
                    self.label.append(color_to_label(folder))
                
    def __len__(self):
        return len(self.label)
        
    def __getitem__(self,index):
        input_img = self.transform(Image.open(self.images[index]).convert('RGB'))
        output_label = torch.tensor(self.label[index])
        return input_img, output_label    

In [ ]:
dataset = data_color(path)

In [ ]:
train_dataset, valid_dataset, test_dataset = random_split(dataset, [int(0.8 * len(dataset)), int(0.1 * len(dataset)), len(dataset) - int(0.1 * len(dataset)) - int(0.8 * len(dataset))])
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torchvision.models as models

color_model = models.resnet18(pretrained=True)
color_model.fc = nn.Linear(color_model.fc.in_features, 3) 
color_model = color_model.to(device)

In [ ]:
import matplotlib.pyplot as plt

train_losses = []
valid_losses = []

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(color_model.parameters(), lr = 1e-4)
softmax = nn.Softmax(dim=-1)
epochs = 15

In [ ]:
for epoch in range(epochs):
    color_model.train()
    predictions = []
    ground_truth = []
    running_loss = 0.0

    dataloader = tqdm(train_dataloader, desc=f"Training on epoch {epoch + 1}...", leave=False)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_hat = color_model(X)
        loss = loss_fn(y_hat, y)
        running_loss += loss.item()

        preds = torch.argmax(softmax(y_hat), dim=1)
        predictions.extend(preds.cpu().numpy())
        ground_truth.extend(y.cpu().numpy())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_train_loss = running_loss / len(train_dataloader)
    train_losses.append(epoch_train_loss)
    print(f"Training epoch {epoch + 1} - Loss: {epoch_train_loss:.4f}")
    print("Classification Report (Train):")
    print(classification_report(ground_truth, predictions))

    # Evaluation
    color_model.eval()
    predictions = []
    ground_truth = []
    running_loss = 0.0

    with torch.no_grad():
        dataloader = tqdm(valid_dataloader, desc=f"Validating on epoch {epoch + 1}...", leave=False)
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            y_hat = color_model(X)
            loss = loss_fn(y_hat, y)
            running_loss += loss.item()

            preds = torch.argmax(softmax(y_hat), dim=1)
            predictions.extend(preds.cpu().numpy())
            ground_truth.extend(y.cpu().numpy())

    epoch_valid_loss = running_loss / len(valid_dataloader)
    valid_losses.append(epoch_valid_loss)
    print(f"Validation epoch {epoch + 1} - Loss: {epoch_valid_loss:.4f}")
    print("Classification Report (Validation):")
    print(classification_report(ground_truth, predictions))

In [ ]:
# Plot after training
plt.figure(figsize=(10, 5))
plt.plot(range(1, 3 + 1), train_losses, label='Training Loss', marker='o')
plt.plot(range(1, 3 + 1), valid_losses, label='Validation Loss', marker='x')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss per Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
color_model.eval()
predictions = []
ground_truth = []
running_loss = 0.0

with torch.no_grad():
    dataloader = tqdm(test_dataloader, desc=f"Testing on epoch {epoch + 1}...", leave=False)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_hat = color_model(X)
        loss = loss_fn(y_hat, y)
        running_loss += loss.item()

        preds = torch.argmax(softmax(y_hat), dim=1)
        predictions.extend(preds.cpu().numpy())
        ground_truth.extend(y.cpu().numpy())

running_loss /= len(test_dataloader)
print(f"Validation epoch {epoch + 1} - Loss: {running_loss:.4f}")
print("Classification Report (Testing):")
print(classification_report(ground_truth, predictions))